In [1]:
from line_solver import *

import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[1].set_number_of_servers(3)

# Default: scheduling is set as FCFS everywhere, routing as Random
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 4, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 0)

node[0].set_service(jobclass[0], Exp(1))
node[0].set_service(jobclass[1], Exp(1))

node[1].set_service(jobclass[0], Exp(1))
node[1].set_service(jobclass[1], Exp(10))

myP = model.init_routing_matrix()
myP.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
myP.set(jobclass[0], jobclass[0], node[1], node[0], 1.0)
myP.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
myP.set(jobclass[1], jobclass[1], node[1], node[0], 1.0)
model.link(myP)

In [3]:
# Solve with multiple solvers
options = SolverQNS.defaultOptions()

solver = np.array([], dtype=object)
solver = np.append(solver, SolverCTMC(model))

# Skip QNS solvers if external qnsolver is not available
if SolverQNS.isAvailable():
    solver = np.append(solver, SolverQNS(model, method='conway'))
    solver = np.append(solver, SolverQNS(model, method='reiser'))
    solver = np.append(solver, SolverQNS(model, method='rolia'))
    solver = np.append(solver, SolverQNS(model, method='zhou'))
else:
    print("QNS solvers skipped: external qnsolver not available")

options = SolverMVA.defaultOptions()
options.config.multiserver = 'softmin'
solver = np.append(solver, SolverMVA(model, options))
options.config.multiserver = 'seidmann'
solver = np.append(solver, SolverMVA(model, options))
solver = np.append(solver, SolverNC(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'SOLVER: {solver[s].get_name()}')
    try:
        AvgTable[s] = solver[s].get_avg_table()
    except Exception as e:
        print(f'  Error: {e}')
        AvgTable[s] = None

SOLVER: SolverCTMC
CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.056065s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  1.9455  1.9455  1.0000  1.0000  1.9455  1.9455
1   Delay   Class2  1.6383  1.6383  1.0000  1.0000  1.6383  1.6383
2  Queue1   Class1  2.0545  0.6485  1.0561  1.0561  1.9455  1.9455
3  Queue1   Class2  0.3617  0.0546  0.2208  0.2208  1.6383  1.6383
SOLVER: QNS
QNS analysis [method: conway, lang: java, env: 21.0.8] completed in 0.082171s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  1.8349  0.3058  1.0000  1.0000  1.8349  1.8349
1   Delay   Class2  1.5168  0.2528  1.0000  1.0000  1.5168  1.5168
2  Queue1   Class1  2.1651  0.6116  1.1799  1.1799  1.8349  1.8349
3  Queue1   Class2  0.4832  0.0506  0.3186  0.3186  1.5168  1.5168
SOLVER: QNS
QNS analysis [method: reiser, lang: java, env: 21.0.8] completed in 0.005280s.
  Station JobClass    QLen    Util   RespT  Resi

Jul 30, 2025 11:14:30 PM jline.io.InputOutputKt line_printf

Jul 30, 2025 11:14:30 PM jline.io.InputOutputKt line_printf

